# Rf 2 0 4_Extrangefit
Addition and convolution: extended maximum likelihood fit with alternate range definition for observed number of events.




**Author:** Clemens Lange, Wouter Verkerke (C++ version)  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:43 AM.</small></i>

In [1]:
import ROOT

Welcome to JupyROOT 6.18/03


Set up component pdfs
---------------------------------------

Declare observable x

In [2]:
x = ROOT.RooRealVar("x", "x", 0, 10)


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create two Gaussian PDFs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and
their parameters

In [3]:
mean = ROOT.RooRealVar("mean", "mean of gaussians", 5)
sigma1 = ROOT.RooRealVar("sigma1", "width of gaussians", 0.5)
sigma2 = ROOT.RooRealVar("sigma2", "width of gaussians", 1)

sig1 = ROOT.RooGaussian("sig1", "Signal component 1", x, mean, sigma1)
sig2 = ROOT.RooGaussian("sig2", "Signal component 2", x, mean, sigma2)

Build Chebychev polynomial p.d.f.

In [4]:
a0 = ROOT.RooRealVar("a0", "a0", 0.5, 0., 1.)
a1 = ROOT.RooRealVar("a1", "a1", -0.2, 0., 1.)
bkg = ROOT.RooChebychev("bkg", "Background", x, ROOT.RooArgList(a0, a1))

Sum the signal components into a composite signal p.d.f.

In [5]:
sig1frac = ROOT.RooRealVar(
    "sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.)
sig = ROOT.RooAddPdf(
    "sig", "Signal", ROOT.RooArgList(sig1, sig2), ROOT.RooArgList(sig1frac))

Construct extended comps with range spec
------------------------------------------------------------------------------

Define signal range in which events counts are to be defined

In [6]:
x.setRange("signalRange", 4, 6)

[#1] INFO:Eval -- RooRealVar::setRange(x) new range named 'signalRange' created with bounds [4,6]


Associated nsig/nbkg as expected number of events with sig/bkg
_in_the_range_ "signalRange"

In [7]:
nsig = ROOT.RooRealVar(
    "nsig", "number of signal events in signalRange", 500, 0., 10000)
nbkg = ROOT.RooRealVar(
    "nbkg", "number of background events in signalRange", 500, 0, 10000)
esig = ROOT.RooExtendPdf(
    "esig", "extended signal p.d.f", sig, nsig, "signalRange")
ebkg = ROOT.RooExtendPdf(
    "ebkg", "extended background p.d.f", bkg, nbkg, "signalRange")

Sum extended components
---------------------------------------------

Construct sum of two extended p.d.f. (no coefficients required)

In [8]:
model = ROOT.RooAddPdf("model", "(g1+g2)+a", ROOT.RooArgList(ebkg, esig))

Sample data, fit model
-------------------------------------------

Generate 1000 events from model so that nsig, come out to numbers <<500
in fit

In [9]:
data = model.generate(ROOT.RooArgSet(x), 1000)

Perform unbinned extended ML fit to data

In [10]:
r = model.fitTo(data, ROOT.RooFit.Extended(ROOT.kTRUE), ROOT.RooFit.Save())
r.Print()

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (sig1,sig2)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (ebkg,esig)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 a1           0.00000e+00  1.00000e-01    0.00000e+00  1.00000e+00
 MINUIT WARNING IN PARAM DEF
 ============== STARTING VALUE IS AT LIMIT.
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE2 IS AT ITS LOWER ALLOWED LIMIT.
 MINUIT WARNING IN PARAMETR
 ============== VARIABLE2 BROUGHT BACK INSIDE LIMITS.
     3 nbkg         5.00000e+02  2.50000e+02    0.00000e+00  1.00000e+04
     4 nsig         5.00000e+02  2.